<a href="https://colab.research.google.com/github/hojink1996/AdversarialData/blob/master/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! rm -r AdversarialData
! git init
! git clone https://github.com/hojink1996/AdversarialData.git

rm: cannot remove 'AdversarialData': No such file or directory
Initialized empty Git repository in /content/.git/
Cloning into 'AdversarialData'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 11137 (delta 2), reused 2 (delta 0), pack-reused 11122
Receiving objects: 100% (11137/11137), 338.61 MiB | 27.32 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (11110/11110), done.


In [1]:
import os
# os.chdir('home')
# os.chdir('..')
! ls

AdversarialData  newresnet50.h5  sample_data


In [0]:
! cat load_single_imagenet.py

In [3]:
from keras.applications import resnet50
from keras import models
from keras.optimizers import SGD
from keras.preprocessing import image
import numpy as np
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import json

from AdversarialData.Tools.retrain_adversarial import generate_new_model
from AdversarialData.Tools.retrain_adversarial import expected_answers
from AdversarialData.Tools.load_single_imagenet import n_arrays_adversarial

# Path to answers
path = 'AdversarialData/val.txt'

# Variables para reentrenar
n_epochs = 100
batch_size = 32

# Load RESNET50
resnet_model = resnet50.ResNet50(weights = 'imagenet')

# Create new model for RESNET50
generate_new_model(resnet_model, 'newresnet50.h5')

# Load model for RESNET50
new_model = models.load_model('newresnet50.h5')

# Expected values
y_expected = expected_answers(path, 5000)
y_train = y_expected

# Set the Gradient Descent
sgd = SGD(lr=5e-3, momentum=0.9, decay=1e-6, nesterov=True)
new_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Get the 2000 first adversarial examples
train_adv = n_arrays_adversarial(0, 5000, 224, 224)
# val_adv = n_arrays_adversarial(11, 20, 224, 224)

# Save best model with Early Stopping
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_acc', patience=5, mode='max')

callbacks_list = [checkpoint, early_stop]

# Fit the model
train_adv = resnet50.preprocess_input(train_adv)
record = new_model.fit(x=train_adv, y=y_train, epochs=n_epochs, shuffle=True, verbose=1, callbacks=callbacks_list,
                       validation_split=0.1)
generate_new_model(new_model, '10000examples100epochs.h5')

# Save the history of training
with open('file.json', 'w') as f:
    json.dump(record.history, f)

102858752/102853048 [==============================] - 4s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1000)              25636712  
Total params: 25,636,712
Trainable params: 25,583,592
Non-trainable params: 53,120
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - 151s 34ms/step - loss: 3.3002 - acc: 0.3036 - val_loss: 4.5491 - val_acc: 0.2160

Epoch 00001: val_acc improved from -inf to 0.21600, saving model to weights.best.hdf5
Epoch 2/100
4500/4500 [==============================] - 133s 29ms/step - loss: 0.7370 - acc: 0.8249 - val_loss: 3.1518 - val_acc: 0.3360

Epoch 00002: val_acc improved from 0.21600 to 0.33600, saving model to weights.best.hdf5
Epoch 3/100
4500/4500 [==============================] - 132s 29ms/step - loss: 0.0978 - acc: 0.9884 - val_loss: 2.7103 - val_acc: 0.4160

Epoch 00003: val_acc improved from 0.33600 to 0.41600, saving model to weights.best.hdf5
Epoch 4/100
4500/4500 [==============================] - 134s 30ms/step - loss: 0.0184 - acc: 1.0000 - val_loss: 2.6522 - val_acc: 0.4180

Epoch 00004: val_acc improved from 0.41600 to 0.41800, saving model to weights.best.hdf5
Epoch 5/100
4500/4500 [=====================

In [4]:
from google.colab import files
files.download('10000examples100epochs.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 41790, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil